In [43]:
import tensorflow as tf
import json
import random 
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from riotwatcher import LolWatcher, ApiError, RiotWatcher
import numpy as np
import sklearn
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
import random
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd


In [3]:
infile = open("buff_x_train.json","r")
X = json.load(infile)
infile.close()
infile = open("buff_y_train.json","r")
Y = json.load(infile)
infile.close()
X_train,X_test,y_train,y_test = train_test_split(X,Y, test_size=0.15,stratify=Y)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))

knn_clf = KNeighborsClassifier(n_neighbors=9)
knn_clf.fit(X_train, y_train)
y_pred = knn_clf.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))

forest_clf = RandomForestClassifier(max_depth=5)
forest_clf.fit(X_train,y_train)
y_pred = forest_clf.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))


0.8240963855421687
[[678  36  50]
 [ 22   2   0]
 [ 37   1   4]]
0.9204819277108434
[[764   0   0]
 [ 24   0   0]
 [ 42   0   0]]
0.9204819277108434
[[764   0   0]
 [ 24   0   0]
 [ 42   0   0]]


In [4]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Reshape(target_shape=(6,),input_shape=(6,)),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units=20,activation='sigmoid'),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units=3,activation='softmax')
    ])

callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join("C:/Users/squat/OneDrive/Desktop/CPSC322/CPSC325/CPSC325_Project", 'ckpt', "{epoch:02d}-{val_loss:.4f}.hdf5"),
            monitor = 'val_loss',
            verbose = 0,
            save_best_only = True,
            save_weights_only = False,
            mode = 'auto',
            save_freq='epoch',
            options=None,
            initial_value_threshold=None,
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=40,
        verbose=1,
        mode='auto',
        min_delta=0.0001,
        cooldown=0,
        min_lr=0,
        )
    ]

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
history = model.fit(X_train,y_train,batch_size=50,callbacks = callbacks, epochs=10,validation_data=(X_test,y_test))
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)
#print(y_pred)
print(y_train.count(0)/len(y_train))
print(y_train.count(1)/len(y_train))
print(y_train.count(2)/len(y_train))
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))

Epoch 1/10
94/94 [==============================] - 3s 11ms/step - loss: 0.3602 - accuracy: 0.9204 - val_loss: 0.3463 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 2/10
94/94 [==============================] - 0s 5ms/step - loss: 0.3357 - accuracy: 0.9204 - val_loss: 0.3308 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 3/10
94/94 [==============================] - 1s 5ms/step - loss: 0.3357 - accuracy: 0.9204 - val_loss: 0.3380 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 4/10
94/94 [==============================] - 0s 5ms/step - loss: 0.3370 - accuracy: 0.9204 - val_loss: 0.3324 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 5/10
94/94 [==============================] - 1s 6ms/step - loss: 0.3363 - accuracy: 0.9204 - val_loss: 0.3309 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 6/10
94/94 [==============================] - 1s 6ms/step - loss: 0.3357 - accuracy: 0.9204 - val_loss: 0.3346 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 7/10
94/94 [==============================] - 0s 5ms/step - loss: 0.3338 - 

In [34]:
# making windowed dataset
windowed_dataset = []
for i in range(len(X)-3):
    temp = X[i:i+3]
    temp = np.array(temp)
    temp = list(temp.flatten())
    windowed_dataset.append(temp)

In [35]:
windowed_y = Y[3:]
print(len(windowed_dataset))
print(len(windowed_y))

5527
5527


In [36]:
X_train,X_test,y_train,y_test = train_test_split(windowed_dataset,windowed_y, test_size=0.15,stratify=windowed_y)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))

knn_clf = KNeighborsClassifier(n_neighbors=9)
knn_clf.fit(X_train, y_train)
y_pred = knn_clf.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))

forest_clf = RandomForestClassifier(max_depth=5)
forest_clf.fit(X_train,y_train)
y_pred = forest_clf.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))


0.8361445783132531
[[690  30  44]
 [ 21   3   0]
 [ 41   0   1]]
0.9204819277108434
[[764   0   0]
 [ 24   0   0]
 [ 42   0   0]]
0.9204819277108434
[[764   0   0]
 [ 24   0   0]
 [ 42   0   0]]


In [38]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Reshape(target_shape=(18,),input_shape=(18,)),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units=20,activation='sigmoid'),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units=3,activation='softmax')
    ])

callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join("C:/Users/squat/OneDrive/Desktop/CPSC322/CPSC325/CPSC325_Project", 'ckpt', "{epoch:02d}-{val_loss:.4f}.hdf5"),
            monitor = 'val_loss',
            verbose = 0,
            save_best_only = True,
            save_weights_only = False,
            mode = 'auto',
            save_freq='epoch',
            options=None,
            initial_value_threshold=None,
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=40,
        verbose=1,
        mode='auto',
        min_delta=0.0001,
        cooldown=0,
        min_lr=0,
        )
    ]

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
history = model.fit(X_train,y_train,batch_size=25,callbacks = callbacks, epochs=10,validation_data=(X_test,y_test))
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)
#print(y_pred)
print(y_train.count(0)/len(y_train))
print(y_train.count(1)/len(y_train))
print(y_train.count(2)/len(y_train))
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))

Epoch 1/10
188/188 [==============================] - 2s 5ms/step - loss: 0.3474 - accuracy: 0.9204 - val_loss: 0.3316 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 2/10
188/188 [==============================] - 1s 4ms/step - loss: 0.3364 - accuracy: 0.9204 - val_loss: 0.3303 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 3/10
188/188 [==============================] - 1s 4ms/step - loss: 0.3427 - accuracy: 0.9204 - val_loss: 0.3320 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 4/10
188/188 [==============================] - 1s 4ms/step - loss: 0.3418 - accuracy: 0.9204 - val_loss: 0.3495 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 5/10
188/188 [==============================] - 1s 4ms/step - loss: 0.3343 - accuracy: 0.9204 - val_loss: 0.3325 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 6/10
188/188 [==============================] - 1s 4ms/step - loss: 0.3402 - accuracy: 0.9204 - val_loss: 0.3617 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 7/10
188/188 [==============================] - 1s 4ms/step - lo

In [41]:
model2 = tf.keras.models.Sequential([
        tf.keras.layers.Reshape(target_shape=(18,1),input_shape=(18,1)),
        tf.keras.layers.Conv1D(5,2,activation='sigmoid',input_shape=(18,1)),
        tf.keras.layers.Conv1D(5,2,activation='sigmoid',input_shape=(17,1)),
        tf.keras.layers.Conv1D(5,2,activation='sigmoid',input_shape=(16,1)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=20,activation='relu'),
        tf.keras.layers.Dense(units=20,activation='relu'),
        tf.keras.layers.Dense(units=10,activation='relu'),
        tf.keras.layers.Dense(units=3,activation='softmax')
    ])

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join('ckpt', "{epoch:02d}-{val_loss:.4f}.hdf5"),
        monitor = 'val_loss',
        verbose = 0,
        save_best_only = True,
        save_weights_only = False,
        mode = 'auto',
        save_freq='epoch',
        options=None,
        initial_value_threshold=None,
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=20,
        verbose=1,
        mode='auto',
        min_delta=0.0001,
        cooldown=0,
        min_lr=0,
        ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=50,
        verbose=1,
        mode='auto',
        baseline=None,
        restore_best_weights=True,
    )
]

model2.summary()

model2.compile(optimizer="adam",loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
history = model2.fit(X_train,y_train,batch_size=50,callbacks = callbacks, epochs=400,validation_data=(X_test,y_test))

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_8 (Reshape)         (None, 18, 1)             0         
                                                                 
 conv1d_6 (Conv1D)           (None, 17, 5)             15        
                                                                 
 conv1d_7 (Conv1D)           (None, 16, 5)             55        
                                                                 
 conv1d_8 (Conv1D)           (None, 15, 5)             55        
                                                                 
 flatten_2 (Flatten)         (None, 75)                0         
                                                                 
 dense_56 (Dense)            (None, 20)                1520      
                                                                 
 dense_57 (Dense)            (None, 20)               

In [62]:
# doing some stats stuff

df = pd.DataFrame()
col_names = ["Trend","KDA","Win","Pick","Role","Ban", "Patched"]
for i in range(len(X[0])):
    df.insert(loc=i,column=col_names[i],value=[row[i] for row in X])

df.insert(loc=6,column = "Patched",value=Y)
print(df)

      Trend   KDA    Win  Pick   Role    Ban  Patched
0      6.52  1.97  49.97  4.43  94.62   2.03        0
1     -0.24  2.56  49.93  4.62  93.47   1.04        0
2      6.51  2.34  48.59  8.16  65.65  12.88        0
3      3.34  2.04  48.57  4.24  34.06  12.88        0
4      0.65  2.23  51.46  4.83  58.01  21.91        0
...     ...   ...    ...   ...    ...    ...      ...
5525   0.00  3.47  50.39  0.33  12.58   1.05        0
5526   0.00  3.05  50.49  2.20  81.61   1.06        0
5527   0.00  2.65  50.20  3.17  87.20   1.68        0
5528   0.00  2.12  47.10  0.38  11.02   1.62        0
5529   0.00  2.12  52.31  3.40  91.87   2.14        0

[5530 rows x 7 columns]


In [68]:
import statsmodels.api as sm

y = df["Patched"]
y = (y-y.min())/2
print(max(y))
x = df[["Trend","KDA","Win","Pick","Role","Ban",]]
x = sm.add_constant(x)

model = sm.OLS(y,x).fit()

print(model.summary())


1.0
                            OLS Regression Results                            
Dep. Variable:                Patched   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.295
Date:                Tue, 09 May 2023   Prob (F-statistic):              0.255
Time:                        21:04:24   Log-Likelihood:                 240.28
No. Observations:                5530   AIC:                            -466.6
Df Residuals:                    5523   BIC:                            -420.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2312      0.085      2.718     

In [69]:
model2 = sm.Logit(y,x).fit()
print(model2.summary())

Optimization terminated successfully.
         Current function value: 0.220016
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                Patched   No. Observations:                 5530
Model:                          Logit   Df Residuals:                     5523
Method:                           MLE   Df Model:                            6
Date:                Tue, 09 May 2023   Pseudo R-squ.:               -0.003211
Time:                        21:04:31   Log-Likelihood:                -1216.7
converged:                       True   LL-Null:                       -1212.8
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0013      1.457      0.001      0.999      -2.855       2.858
Trend          0.0094      0.

In [75]:
x = df[["Win","Pick","Ban",]]
x = x.values.tolist()
X_train,X_test,y_train,y_test = train_test_split(x,Y, test_size=0.15,stratify=Y)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))

knn_clf = KNeighborsClassifier(n_neighbors=9)
knn_clf.fit(X_train, y_train)
y_pred = knn_clf.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))

forest_clf = RandomForestClassifier(max_depth=5)
forest_clf.fit(X_train,y_train)
y_pred = forest_clf.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))

0.844578313253012
[[696  28  40]
 [ 23   0   1]
 [ 36   1   5]]
0.9204819277108434
[[764   0   0]
 [ 24   0   0]
 [ 42   0   0]]
0.9204819277108434
[[764   0   0]
 [ 24   0   0]
 [ 42   0   0]]


In [76]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Reshape(target_shape=(3,),input_shape=(3,)),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units=20,activation='sigmoid'),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units = 20,activation='sigmoid'),
        tf.keras.layers.Dense(units=3,activation='softmax')
    ])

callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join("C:/Users/squat/OneDrive/Desktop/CPSC322/CPSC325/CPSC325_Project", 'ckpt', "{epoch:02d}-{val_loss:.4f}.hdf5"),
            monitor = 'val_loss',
            verbose = 0,
            save_best_only = True,
            save_weights_only = False,
            mode = 'auto',
            save_freq='epoch',
            options=None,
            initial_value_threshold=None,
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=40,
        verbose=1,
        mode='auto',
        min_delta=0.0001,
        cooldown=0,
        min_lr=0,
        )
    ]

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
history = model.fit(X_train,y_train,batch_size=50,callbacks = callbacks, epochs=20,validation_data=(X_test,y_test))
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)
#print(y_pred)
print(y_train.count(0)/len(y_train))
print(y_train.count(1)/len(y_train))
print(y_train.count(2)/len(y_train))
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))

Epoch 1/20
94/94 [==============================] - 1s 8ms/step - loss: 0.3707 - accuracy: 0.9204 - val_loss: 0.3322 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 2/20
94/94 [==============================] - 0s 4ms/step - loss: 0.3340 - accuracy: 0.9204 - val_loss: 0.3335 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 3/20
94/94 [==============================] - 1s 6ms/step - loss: 0.3350 - accuracy: 0.9204 - val_loss: 0.3323 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 4/20
94/94 [==============================] - 1s 8ms/step - loss: 0.3356 - accuracy: 0.9204 - val_loss: 0.3334 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 5/20
94/94 [==============================] - 0s 5ms/step - loss: 0.3388 - accuracy: 0.9204 - val_loss: 0.3449 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 6/20
94/94 [==============================] - 0s 5ms/step - loss: 0.3374 - accuracy: 0.9204 - val_loss: 0.3298 - val_accuracy: 0.9205 - lr: 0.1000
Epoch 7/20
94/94 [==============================] - 0s 4ms/step - loss: 0.3367 - a